In [107]:
import pandas as pd

In [108]:
df1 = pd.read_csv('Full_Country_Covid.csv')
df2 = pd.read_csv('full_data.csv')
pd.set_option('display.max_columns', None)

/Users/vathanahim/opt/anaconda3/envs/MSIA/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [109]:
#Remove uneeded column
df1 = df1.drop(['CountryCode','RegionName', 'RegionCode'], axis=1)
#convert date to appropriate formatt
df1['Date'] = pd.to_datetime(df1['Date'], format='%Y%m%d')

##### Clean data for united states because it's the only country that has data for specific states so we have to take average and group it by country and date then clean df1 after 

In [110]:
#select only United States data
df1_united_states = df1.loc[df1['CountryName']=='United States']
df1_united_states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14151 entries, 47793 to 61943
Data columns (total 39 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   CountryName                            14151 non-null  object        
 1   Date                                   14151 non-null  datetime64[ns]
 2   C1_School closing                      13515 non-null  float64       
 3   C1_Flag                                9671 non-null   float64       
 4   C2_Workplace closing                   13496 non-null  float64       
 5   C2_Flag                                9740 non-null   float64       
 6   C3_Cancel public events                13496 non-null  float64       
 7   C3_Flag                                9709 non-null   float64       
 8   C4_Restrictions on gatherings          13496 non-null  float64       
 9   C4_Flag                                8699 non-null   fl

In [111]:
#get list of columns that we need to take average of 
df1_united_states_col = [i for i in df1_united_states.columns]
df1_united_states_col[2:]

['C1_School closing',
 'C1_Flag',
 'C2_Workplace closing',
 'C2_Flag',
 'C3_Cancel public events',
 'C3_Flag',
 'C4_Restrictions on gatherings',
 'C4_Flag',
 'C5_Close public transport',
 'C5_Flag',
 'C6_Stay at home requirements',
 'C6_Flag',
 'C7_Restrictions on internal movement',
 'C7_Flag',
 'C8_International travel controls',
 'E1_Income support',
 'E1_Flag',
 'E2_Debt/contract relief',
 'E3_Fiscal measures',
 'E4_International support',
 'H1_Public information campaigns',
 'H1_Flag',
 'H2_Testing policy',
 'H3_Contact tracing',
 'H4_Emergency investment in healthcare',
 'H5_Investment in vaccines',
 'M1_Wildcard',
 'StringencyIndex',
 'StringencyIndexForDisplay',
 'StringencyLegacyIndex',
 'StringencyLegacyIndexForDisplay',
 'GovernmentResponseIndex',
 'GovernmentResponseIndexForDisplay',
 'ContainmentHealthIndex',
 'ContainmentHealthIndexForDisplay',
 'EconomicSupportIndex',
 'EconomicSupportIndexForDisplay']

In [112]:
#groupy by and average it 
df1_united_states = df1_united_states.groupby(['CountryName','Date'])[df1_united_states_col[2:]].mean().reset_index()

In [113]:
#delete old united states data from overall dataframe and add new one
df1 = df1[df1.CountryName != 'United States']
df1 = df1.append(df1_united_states)

In [114]:
df1_clean = df1[['CountryName','Date','StringencyIndex','GovernmentResponseIndex','ContainmentHealthIndex','EconomicSupportIndex']]
df1_clean = df1_clean.dropna()

In [126]:
#convert date object in dataframe back to string format 
df1_clean['Date'] =df1_clean['Date'].dt.strftime('%Y-%m-%d')
df1_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47650 entries, 0 to 254
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CountryName              47650 non-null  object 
 1   Date                     47650 non-null  object 
 2   StringencyIndex          47650 non-null  float64
 3   GovernmentResponseIndex  47650 non-null  float64
 4   ContainmentHealthIndex   47650 non-null  float64
 5   EconomicSupportIndex     47650 non-null  float64
dtypes: float64(4), object(2)
memory usage: 2.5+ MB


#### clean df2 by formatting dates and caculating fatality ratio

In [127]:
df2['fatality_ratio'] = df2['total_deaths']/df2['total_cases']

In [128]:
df1_country_list = df1_clean['CountryName'].values.tolist()

In [129]:
#filter for countries that only appear in policy data and select only needed rows
df2_clean = df2.location.isin(df1_country_list)
df2_clean = df2[['date','total_cases','total_deaths','fatality_ratio', 'location']]
df2_clean = df2_clean.dropna()
df2_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38030 entries, 56 to 41367
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            38030 non-null  object 
 1   total_cases     38030 non-null  float64
 2   total_deaths    38030 non-null  float64
 3   fatality_ratio  38030 non-null  float64
 4   location        38030 non-null  object 
dtypes: float64(3), object(2)
memory usage: 1.7+ MB


### Data storage

In [146]:
import sqlite3

In [147]:
db = "proj.db"
conn = sqlite3.connect(db)
print(conn)

In [148]:
#creating database with appropriate fields

conn = sqlite3.connect(db)
crsr = conn.cursor()
crsr.execute("DROP TABLE IF EXISTS 'POLICY_DATA'")
print("Table Dropped")
conn.close()


conn = sqlite3.connect(db)
crsr = conn.cursor()
crsr.execute("DROP TABLE IF EXISTS 'COVID_DATA'")
print("Table Dropped")
conn.close()

Table Dropped
Table Dropped


In [149]:
df1_list = df1_clean.values.tolist()
df1_list

[['Aruba', '2020-01-01', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-02', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-03', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-04', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-05', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-06', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-07', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-08', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-09', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-10', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-11', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-12', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-13', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-14', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-15', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-16', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-17', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-18', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-19', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-20', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-21', 0.0, 0.0, 0.0, 0.0],
 ['Aruba', '2020-01-22', 0.0, 0.0,

In [150]:
df2_list = df2_clean.values.tolist()
df2_list

[['2020-02-25', 1.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-02-26', 1.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-02-27', 1.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-02-28', 1.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-02-29', 1.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-01', 1.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-02', 1.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-08', 4.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-11', 7.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-15', 10.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-16', 16.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-17', 21.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-18', 22.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-19', 22.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-20', 22.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-21', 24.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-22', 24.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-23', 34.0, 0.0, 0.0, 'Afghanistan'],
 ['2020-03-24', 40.0, 1.0, 0.025, 'Afghanistan'],
 ['2020-03-25', 42.0, 1.0, 0.023809523809523808, 'Afghanistan'],
 ['2020-03-26', 75.0, 1.0, 0.0

In [152]:
#create table for POLICY DATA and COVID DATA
conn = sqlite3.connect(db)
crsr = conn.cursor()
try:
    crsr.execute("""
    CREATE TABLE POLICY_DATA (
    Country_Name TEXT,
    Date datetime,
    Stringency_Index FLOAT DEFAULT 0,
    Government_Response_Index FLOAT DEFAULT 0,
    Containment_Health_Index FLOAT DEFAULT 0,
    Economic_Support_Index FLOAT DEFAULT 0
    );
    """)
    crsr.execute("""
    CREATE TABLE COVID_DATA (
    Date datetime,
    Total_Case FLOAT DEFAULT 0,
    Total_Death FLOAT DEFAULT 0,
    Fatality_Ratio FLOAT DEFAULT 0,
    Location TEXT
    );""")
    print("Success Creation")
except Exception as e:
    print(str(e))
    print("Failed Creation")
finally:
    conn.close()

Success Creation


In [153]:
#insert data policy into table
conn = sqlite3.connect(db)
crsr = conn.cursor()
try:
    crsr.executemany("""
    INSERT INTO POLICY_DATA (Country_Name, Date, Stringency_Index, Government_Response_Index,
    Containment_Health_Index, Economic_Support_Index)
    VALUES (?,?,?,?,?,?)
    """, df1_list)
    conn.commit()
    print("Insert Successful")
except Exception as e:
    print(str(e))
    print("Insert Failed")
finally:
    conn.close()
    
#insert data covid into table
conn = sqlite3.connect(db)
crsr = conn.cursor()
try:
    crsr.executemany("""
    INSERT INTO COVID_DATA (Date, Total_Case, Total_Death, Fatality_Ratio, Location)
    VALUES (?,?,?,?,?)
    """, df2_list)
    conn.commit()
    print("Insert Successful")
except Exception as e:
    print(str(e))
    print("Insert Failed")
finally:
    conn.close()

Insert Successful
Insert Successful


In [154]:
#creating table that contains total data
conn = sqlite3.connect(db)
crsr = conn.cursor()
crsr.execute("DROP TABLE IF EXISTS `Totaldata`")
print("Table dropped")
conn.close()

Table dropped


In [156]:
#Join two tables together into one
conn = sqlite3.connect(db)
crsr = conn.cursor()
crsr.execute("""
CREATE TABLE Totaldata as SELECT * FROM (
SELECT * FROM COVID_DATA
INNER JOIN POLICY_DATA ON POLICY_DATA.Date = COVID_DATA.Date AND POLICY_DATA.Country_Name = COVID_DATA.Location
)
""")
conn.close()

In [157]:
conn = sqlite3.connect(db)
sql = "Select * FROM Totaldata"
df_total = pd.read_sql(sql, conn)
df_total

,Date,Total_Case,Total_Death,Fatality_Ratio,Location,Country_Name,Date:1,Stringency_Index,Government_Response_Index,Containment_Health_Index,Economic_Support_Index
0,2020-02-25,1.0,0.0,0.000000,Afghanistan,Afghanistan,2020-02-25,8.33,12.18,14.39,0.0
1,2020-02-26,1.0,0.0,0.000000,Afghanistan,Afghanistan,2020-02-26,8.33,12.18,14.39,0.0
2,2020-02-27,1.0,0.0,0.000000,Afghanistan,Afghanistan,2020-02-27,8.33,12.18,14.39,0.0
3,2020-02-28,1.0,0.0,0.000000,Afghanistan,Afghanistan,2020-02-28,8.33,12.18,14.39,0.0
4,2020-02-29,1.0,0.0,0.000000,Afghanistan,Afghanistan,2020-02-29,8.33,12.18,14.39,0.0
...,...,...,...,...,...,...,...,...,...,...,...
32025,2020-09-01,6497.0,202.0,0.031091,Zimbabwe,Zimbabwe,2020-09-01,80.56,66.03,73.48,25.0
32026,2020-09-02,6559.0,203.0,0.030950,Zimbabwe,Zimbabwe,2020-09-02,80.56,66.03,73.48,25.0
32027,2020-09-03,6638.0,206.0,0.031033,Zimbabwe,Zimbabwe,2020-09-03,80.56,66.03,73.48,25.0
32028,2020-09-04,6678.0,206.0,0.030848,Zimbabwe,Zimbabwe,2020-09-04,80.56,66.03,73.48,25.0
